# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
import sys
sys.path.append("..")

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = '../WeatherPy/output_data/weather_data.csv'
weather_df = pd.read_csv(file_path)

weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nikolskoye,59.70,30.79,33.80,86.0,75.0,4.47,RU,1.615915e+09
1,1,plettenberg bay,-34.05,23.37,65.44,78.0,76.0,11.65,ZA,1.615915e+09
2,2,sao filipe,14.90,-24.50,72.66,73.0,0.0,13.85,CV,1.615915e+09
3,3,nanortalik,60.14,-45.24,26.92,95.0,100.0,19.01,GL,1.615915e+09
4,4,tumaco,1.80,-78.82,80.65,82.0,95.0,5.91,CO,1.615915e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(weather_df[['Lat','Lng']],
                                 weights=weather_df['Humidity'],
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

A max temperature lower than 80F degrees but higher than 70F.


Wind speed less than 10 mph.


Zero cloudiness.

In [5]:
# get spots within temperature range
vacation_spots = weather_df.loc[(weather_df['Max Temp'] < 80) & (weather_df['Max Temp'] > 70)]
# get spots with low winds
vacation_spots = vacation_spots.loc[vacation_spots['Wind Speed'] < 10]
# get spots with 0 clouds
vacation_spots = vacation_spots.loc[vacation_spots['Cloudiness'] == 0]

In [6]:
vacation_spots.reset_index(drop=True, inplace=True)
vacation_spots.drop(columns='Unnamed: 0', inplace=True)
vacation_spots.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,dubai,25.26,55.30,77.00,57.0,0.0,5.75,AE,1.615915e+09
1,anantapur,14.68,77.60,79.63,31.0,0.0,9.51,IN,1.615915e+09
2,sur,22.57,59.53,75.11,63.0,0.0,4.23,OM,1.615915e+09
3,chodavaram,17.83,82.95,73.71,80.0,0.0,2.30,IN,1.615915e+09
4,victoria,22.29,114.16,73.99,85.0,0.0,5.70,HK,1.615915e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame({
    'Hotel Name': '',
    'City': '',
    'Lat': [],
    'Lng': [],
    'Country': ''
})

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
for row in range(0,len(vacation_spots)):
    lat = vacation_spots.loc[row,['Lat']].values
    lng = vacation_spots.loc[row,['Lng']].values

    query_params = {
         'key': g_key,
         'location': f'{lat[0]},{lng[0]}',
         'radius':5000,
         'type': 'hotel'   
     }

    response_json = requests.get(base_url, params=query_params).json()
    result = response_json['results'][0]

    hotel_df.loc[row,'Hotel Name'] = result['name']
    hotel_df.loc[row,'City'] = vacation_spots.loc[row, 'City']
    hotel_df.loc[row,'Lat'] = lat[0]
    hotel_df.loc[row,'Lng'] = lng[0]
    hotel_df.loc[row,'Country'] = vacation_spots.loc[row, 'Country']

hotel_df

,Hotel Name,City,Lat,Lng,Country
0,Dubai,dubai,25.26,55.30,AE
1,Anantapur,anantapur,14.68,77.60,IN
2,Sur,sur,22.57,59.53,OM
3,Venkannapalem,chodavaram,17.83,82.95,IN
4,Hong Kong,victoria,22.29,114.16,HK
5,Thái Bình,thai binh,20.45,106.33,VN
6,Doha,doha,25.29,51.53,QA
7,Iranshahr,iranshahr,27.20,60.68,IR
8,Pisco,pisco,-13.70,-76.22,PE
9,Visakhapatnam,yarada,17.65,83.27,IN


In [8]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Hotel Name  11 non-null     object 
 1   City        11 non-null     object 
 2   Lat         11 non-null     float64
 3   Lng         11 non-null     float64
 4   Country     11 non-null     object 
dtypes: float64(2), object(3)
memory usage: 848.0+ bytes


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(locations, scale=2, info_box_content=hotel_info)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))